# Onion Shelf Life Prediction Model Training
## Computer Vision-Based Quality Assessment System

**Project Overview:**
This notebook documents the complete training pipeline for an onion shelf life prediction model using computer vision and deep learning. The model classifies onions into 4 quality classes based on visual features extracted from images.

**Model Classes:**
- Class 0: Severe deterioration (0 days shelf life)
- Class 1: Poor quality (5-10 days shelf life)
- Class 2: Fair quality (15-19 days shelf life)
- Class 3: Excellent quality (29-37 days shelf life)

**Author:** Onion Quality Assessment Team  
**Date:** February 2026  
**Framework:** TensorFlow.js / Keras

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from datetime import datetime

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"OpenCV version: {cv2.__version__}")

## 2. Feature Extraction Functions

These functions extract visual features from onion images using computer vision techniques. Features include:
- Physical dimensions (diameter, size classification)
- Black spots count (dark pixel detection)
- Surface texture score (local variance analysis)
- Skin condition score (brightness/saturation analysis)
- Damage indicators (bruises, cuts, lesions)
- Sprouting detection (green pixel ratio)
- Color analysis (brightness, saturation, uniformity)
- Firmness indicators
- Root condition assessment

In [ ]:
class OnionFeatureExtractor:
    """Extract visual features from onion images for quality assessment"""
    
    def __init__(self):
        self.features = {}
    
    def is_onion_pixel(self, r, g, b):
        """Check if pixel color matches onion characteristics"""
        brightness = (r + g + b) / 3
        
        # Brown/Golden/Yellow onion
        if 80 <= r <= 230 and 50 <= g <= 200 and 20 <= b <= 130 and r > b + 20:
            return True
        # White onion
        if r >= 170 and g >= 170 and b >= 170:
            return True
        # Purple/Red onion
        if 80 <= r <= 180 and 70 <= b <= 170 and r > g - 10 and b > g - 30:
            return True
        
        return False
    
    def estimate_dimensions(self, image):
        """Estimate onion dimensions from image"""
        height, width = image.shape[:2]
        min_x, max_x = width, 0
        min_y, max_y = height, 0
        onion_pixels = 0
        
        for y in range(height):
            for x in range(width):
                b, g, r = image[y, x]
                
                if self.is_onion_pixel(r, g, b):
                    onion_pixels += 1
                    min_x = min(min_x, x)
                    max_x = max(max_x, x)
                    min_y = min(min_y, y)
                    max_y = max(max_y, y)
        
        pixel_width = max_x - min_x
        pixel_height = max_y - min_y
        
        # Estimate real dimensions (assuming average onion is ~75mm)
        scale_factor = 75 / max(pixel_width, pixel_height) if max(pixel_width, pixel_height) > 0 else 1
        
        diameter_mm = round(max(pixel_width, pixel_height) * scale_factor, 1)
        
        # Size classification
        if diameter_mm < 50:
            size_class = 'Small'
        elif diameter_mm < 75:
            size_class = 'Medium'
        elif diameter_mm < 100:
            size_class = 'Large'
        else:
            size_class = 'Extra Large'
        
        return {
            'diameter_mm': diameter_mm,
            'size_class': size_class,
            'length_mm': round(pixel_height * scale_factor, 1),
            'width_mm': round(pixel_width * scale_factor, 1)
        }
    
    def count_black_spots(self, image):
        """Count black spots on onion surface"""
        height, width = image.shape[:2]
        dark_spot_pixels = 0
        onion_pixels = 0
        
        for y in range(height):
            for x in range(width):
                b, g, r = image[y, x]
                
                if self.is_onion_pixel(r, g, b):
                    onion_pixels += 1
                    brightness = (r + g + b) / 3
                    
                    # Dark spots: significantly darker than normal onion
                    if 20 < brightness < 80:
                        dark_spot_pixels += 1
        
        if onion_pixels == 0:
            return 0
        
        dark_ratio = dark_spot_pixels / onion_pixels
        
        # Map ratio to spot count (0-21 range)
        if dark_ratio < 0.02:
            spot_count = int(dark_ratio * 100)
        elif dark_ratio < 0.08:
            spot_count = 2 + int((dark_ratio - 0.02) * 100)
        else:
            spot_count = 8 + int((dark_ratio - 0.08) * 150)
        
        return min(21, max(0, spot_count))
    
    def analyze_surface_texture(self, image):
        """Analyze surface texture (1=smooth to 4=very soft)"""
        height, width = image.shape[:2]
        total_variance = 0
        samples = 0
        
        # Calculate local variance as texture indicator
        for y in range(10, height - 10, 10):
            for x in range(10, width - 10, 10):
                patch = image[y-5:y+5, x-5:x+5]
                variance = np.var(patch)
                total_variance += variance
                samples += 1
        
        avg_variance = total_variance / samples if samples > 0 else 0
        
        # Map variance to texture score
        if avg_variance < 100:
            return 1  # Smooth
        elif avg_variance < 300:
            return 2  # Slightly wrinkled
        elif avg_variance < 600:
            return 3  # Wrinkled
        else:
            return 4  # Very soft/deteriorated
    
    def analyze_skin_condition(self, image):
        """Analyze skin condition (1=excellent to 5=poor)"""
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Analyze saturation and value
        avg_saturation = np.mean(hsv[:, :, 1])
        avg_value = np.mean(hsv[:, :, 2])
        
        # Higher saturation and value = better condition
        condition_score = (avg_saturation + avg_value) / 2
        
        if condition_score > 180:
            return 1  # Excellent
        elif condition_score > 140:
            return 2  # Good
        elif condition_score > 100:
            return 3  # Fair
        elif condition_score > 60:
            return 4  # Poor
        else:
            return 5  # Very poor
    
    def extract_all_features(self, image_path):
        """Extract all features from an onion image"""
        image = cv2.imread(image_path)
        
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        dimensions = self.estimate_dimensions(image)
        black_spots = self.count_black_spots(image)
        texture = self.analyze_surface_texture(image)
        skin_condition = self.analyze_skin_condition(image)
        
        return {
            **dimensions,
            'black_spots_count': black_spots,
            'surface_texture_score': texture,
            'skin_condition_score': skin_condition
        }

## 3. Model Architecture

Build a neural network for shelf life classification

In [ ]:
def create_shelf_life_model(input_dim):
    """Create neural network model for shelf life prediction"""
    model = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(16, activation='relu'),
        layers.Dense(4, activation='softmax')  # 4 quality classes
    ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

## 4. Training Pipeline

Load data, train model, and evaluate performance

In [ ]:
# Load dataset
df = pd.read_csv('onion_shelf_life_dataset.csv')

# Prepare features and labels
feature_columns = [
    'diameter_mm', 'black_spots_count', 'surface_texture_score',
    'skin_condition_score', 'has_bruises', 'has_cuts', 'has_lesions'
]

X = df[feature_columns].values

# Convert shelf life to classes
def shelf_life_to_class(days):
    if days < 5:
        return 0  # Severe
    elif days < 15:
        return 1  # Poor
    elif days < 25:
        return 2  # Fair
    else:
        return 3  # Excellent

y = df['estimated_shelf_life_days'].apply(shelf_life_to_class).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train model
model = create_shelf_life_model(X_train.shape[1])

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)

# Evaluate
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy:.4f}")

# Predictions
y_pred = np.argmax(model.predict(X_test), axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Severe', 'Poor', 'Fair', 'Excellent']))

## 5. Save Model for TensorFlow.js

In [ ]:
# Save model in TensorFlow.js format
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, 'my_model')
print("Model saved for TensorFlow.js")